 ## Exercise 39bis - Pollution analysis V2

In [1]:
inputPath  = "/data/students/bigdata-01QYD/ex_data/Ex39bis/data/"
outputPath = "out_Ex39/" 

# read from input and load into the RDD
logRDD = sc.textFile(inputPath)
logRDD.collect()

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5',
 's3,2016-01-03,12.5']

__Output__ : HDFS file containing one line for each sensor. Each line contains a sensorId and the list of dates with a PM10 
values greater than 50 for that sensor. Consider only the sensors associated at least one time with a PM10 value greater than 5

** bonus : Also the sensors which have never been associated with a PM10 values greater than 50 must be included in the result (with an empty set)

In [3]:
# step 1 = filter
filteredRDD = logRDD.filter(lambda x : float(x.split(",")[2]) > 50)

# step 2-3 = map - groupByKey 
# NB the output of the groupByKey is an iterator, so you need to map it
pairsRDD = filteredRDD.map(lambda x : (x.split(",")[0], x.split(",")[1]) )\
                      .groupByKey()\
                      .mapValues(lambda dates : list(dates))

pairsRDD.collect()

[('s1', ['2016-01-02', '2016-01-03']), ('s2', ['2016-01-03'])]

In [10]:
# we get the values that are not present in pairsRDD
# we map it into a key value pair, with an empty list
remainingRDD = logRDD.map(lambda pair : pair.split(",")[0])\
                     .subtract(pairsRDD.keys())\
                     .map(lambda record : (record, []))
remainingRDD.collect()

[('s3', [])]

In [11]:
# combine these RDD with UNION

finalRDD = pairsRDD.union(remainingRDD)

finalRDD.collect()

[('s1', ['2016-01-02', '2016-01-03']), ('s2', ['2016-01-03']), ('s3', [])]

In [ ]:
finalRDD.saveAsTextFile(outputPath)